In [ ]:
! pip install xgboost
! pip install imblearn
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.svm import SVC
data = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
data = data.dropna()
test = test.dropna()
X = data.drop(['Segmentation','ID'],axis=1)
y = data['Segmentation']

X_test = test.drop(['Segmentation','ID'],axis=1)
y_test = test['Segmentation']
X=pd.get_dummies(X,drop_first=True)
X_test = pd.get_dummies(X_test,drop_first=True)
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
label_encoder = preprocessing.LabelEncoder()
scaler=preprocessing.StandardScaler()
X=scaler.fit_transform(X)
X_test=scaler.fit_transform(X_test)

y= label_encoder.fit_transform(y)
y_test = label_encoder.fit_transform(y_test)

# Define the search space
param_grid = { 
    # Percentage of columns to be randomly samples for each tree.
    "colsample_bytree": [ 0.3, 0.5 , 0.8 ],
    # reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [0, 0.5, 1, 5],
    # reg_lambda provides l2 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [0, 0.5, 1, 5]
    }
# Set up score
scoring = ['recall']
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
model_1= XGBClassifier(learning_rate=0.02, n_estimators=600)
grid_search = GridSearchCV(estimator=model_1, 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           refit='recall', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)
# Fit grid search
grid_result = grid_search.fit(X, y)

grid_search.fit(X,y)
y_pred = grid_search.predict(X_test)
print(classification_report(y_test,y_pred))